# SQL Interview Questions - Lab

## Introduction

In this lab, we'll test our SQL skills against some real-world interview questions from major companies!

## Objectives

You will be able to:

* Write SQL queries to filter and order results
* Decide and perform whichever type of join is best for retrieving desired data
* Write subqueries to decompose complex queries

## Getting Started

In this lab, we'll see four different interview questions that test your SQL knowledge. We didn't write these questions -- instead, we found them out in the real-world. These are questions that have been used in the past by major technology companies such as Facebook, Amazon, and Twitter. Our goal here isn't to memorize the questions or anything like that -- after all, it's extremely unlikely that these questions are still in use, now that they've become publicly available on the interwebs. Instead, our goal is to treat these questions as if they are the real thing, and give us some insight into the types of questions we'll need to be able to answer in order pass an interview involving SQL. 


### A Note on Answering These Questions

Since these are interview questions, they'll almost always be posed as hypotheticals. This means that you won't have a real database to work with and test your code on. This also means that there are multiple different solutions to any given problem listed here. Be sure to doublecheck the code you write for bugs and errors. It's much harder to write bug-free code when you aren't able to test it against a database!

If these questions seem hard, that's normal. These are real questions that have been reported to online forums from job seekers at major companies. Obviously, it's unlikely that they're still in use at these companies, but they still represent a great way for us to test our skills against the kinds of questions we can expect to be asked in an interview!

## Question 1

From Facebook:

Assume we have a table of employee information, which includes salary information. Write a query to find the names and salaries of the top 5 highest paid employees, in descending order.

In [6]:
# Your code here
import sqlite3

import sqlite3

# Create an in-memory SQLite database
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# 1. Create the table
cursor.execute("""
CREATE TABLE employees (
    id INTEGER PRIMARY KEY,
    name TEXT,
    salary REAL
);
""")

# 2. Insert some example data
cursor.executemany("""
INSERT INTO employees (name, salary)
VALUES (?, ?)
""", [
    ("Alice", 90000),
    ("Bob", 120000),
    ("Charlie", 110000),
    ("Diana", 95000),
    ("Ethan", 130000),
    ("Fiona", 125000),
    ("George", 95000)
])

conn.commit()

# 3. Run the top 5 salaries query
query = """
SELECT
    name,
    salary
FROM employees
ORDER BY salary DESC
LIMIT 5;
"""

cursor.execute(query)
results = cursor.fetchall()

# Show results
for row in results:
    print(row)


('Ethan', 130000.0)
('Fiona', 125000.0)
('Bob', 120000.0)
('Charlie', 110000.0)
('Diana', 95000.0)


## Question 2

From Amazon:

Assume we have two SQL tables: `authors` and `books`. The authors table has a few million rows, and looks like this: 

| author_name | book_name |
|:-----------:|:---------:|
|   author_1  |   book_1  |
|   author_1  |   book_2  |
|   author_2  |   book_3  |
|   author_2  |   book_4  |
|   author_2  |   book_5  |
|   author_3  |   book_6  |

The books dataset also has a few million rows, and looks like this:

| book_name | copies_sold |
|:---------:|:-----------:|
|   book_1  |    10000    |
|   book_2  |     2575    |
|   book_3  |    60000    |
|   book_4  |    98000    |
|   book_5  |     5250    |
|   book_6  |    19775    |

Write an SQL query that shows the top 3 authors who sold the most total books. 

In [8]:
# Your code here
import sqlite3

# Create an in-memory SQLite database
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Create and populate the tables
cursor.execute("""
CREATE TABLE authors (
    author_name TEXT,
    book_name TEXT
);
""")

cursor.execute("""
CREATE TABLE books (
    book_name TEXT,
    copies_sold INTEGER
);
""")

cursor.executemany("INSERT INTO authors VALUES (?, ?)", [
    ("author_1", "book_1"),
    ("author_1", "book_2"),
    ("author_2", "book_3"),
    ("author_2", "book_4"),
    ("author_2", "book_5"),
    ("author_3", "book_6")
])

cursor.executemany("INSERT INTO books VALUES (?, ?)", [
    ("book_1", 10000),
    ("book_2", 2575),
    ("book_3", 60000),
    ("book_4", 98000),
    ("book_5", 5250),
    ("book_6", 19775)
])

# Now run the query
query = """
SELECT
    a.author_name,
    SUM(b.copies_sold) AS total_copies_sold
FROM authors a
JOIN books b
    ON a.book_name = b.book_name
GROUP BY a.author_name
ORDER BY total_copies_sold DESC
LIMIT 3;
"""

cursor.execute(query)
results = cursor.fetchall()
for row in results:
    print(row)


('author_2', 163250)
('author_3', 19775)
('author_1', 12575)


## Question 3

From Amazon:

Assume you have two tables, `customers` and `orders`. Write a SQL query to select all customers who purchased at least 2 items on two separate days. 

In [10]:
# Your code here
import sqlite3

# Create in-memory SQLite database
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# 1. Create tables
cursor.execute("""
CREATE TABLE customers (
    customer_id INTEGER PRIMARY KEY,
    customer_name TEXT
);
""")

cursor.execute("""
CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    order_date TEXT,
    quantity INTEGER
);
""")

# 2. Insert sample customers
cursor.executemany("INSERT INTO customers (customer_id, customer_name) VALUES (?, ?)", [
    (1, "Alice"),
    (2, "Bob"),
    (3, "Charlie")
])

# 3. Insert sample orders
cursor.executemany("INSERT INTO orders (customer_id, order_date, quantity) VALUES (?, ?, ?)", [
    (1, "2025-08-01", 2),  # Alice, 2 items day 1
    (1, "2025-08-02", 3),  # Alice, 3 items day 2
    (1, "2025-08-03", 1),  # Alice, 1 item day 3 (not counted)
    (2, "2025-08-01", 2),  # Bob, 2 items day 1
    (2, "2025-08-02", 1),  # Bob, only 1 item day 2 (not counted)
    (2, "2025-08-03", 2),  # Bob, 2 items day 3
    (3, "2025-08-01", 1),  # Charlie, never meets >=2 items
    (3, "2025-08-02", 1)
])

conn.commit()

# 4. Query: customers who purchased ≥2 items on ≥2 separate days
query = """
SELECT c.customer_id, c.customer_name
FROM customers c
JOIN (
    SELECT customer_id
    FROM (
        SELECT
            customer_id,
            order_date,
            SUM(quantity) AS total_items
        FROM orders
        GROUP BY customer_id, order_date
        HAVING SUM(quantity) >= 2
    ) days_with_2plus
    GROUP BY customer_id
    HAVING COUNT(DISTINCT order_date) >= 2
) qualified_customers
ON c.customer_id = qualified_customers.customer_id;
"""

cursor.execute(query)
results = cursor.fetchall()

# 5. Display results
for row in results:
    print(row)


(1, 'Alice')
(2, 'Bob')


## Question 4

From Twitter:

A company uses 2 data tables, `Employee` and `Department`, to store data about its employees and departments. 

Table Name: Employee   
Attributes:   
ID Integer,   
NAME String,   
SALARY Integer,   
DEPT_ID Integer   

Table Name: Department   
Attributes:   
DEPT_ID Integer,   
NAME String,   
LOCATION String   

Write a query to print the respective Department Name and number of employees for all departments in the Department table (even unstaffed ones). 

Sort your result in descending order of employees per department; if two or more departments have the same number of employees, then sort those departments alphabetically by Department Name.

In [12]:
# Your code here
import sqlite3

# Create in-memory SQLite database
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# 1. Create Department table
cursor.execute("""
CREATE TABLE Department (
    DEPT_ID INTEGER PRIMARY KEY,
    NAME TEXT,
    LOCATION TEXT
);
""")

# 2. Create Employee table
cursor.execute("""
CREATE TABLE Employee (
    ID INTEGER PRIMARY KEY,
    NAME TEXT,
    SALARY INTEGER,
    DEPT_ID INTEGER
);
""")

# 3. Insert sample Department data
cursor.executemany("INSERT INTO Department (DEPT_ID, NAME, LOCATION) VALUES (?, ?, ?)", [
    (1, "Engineering", "New York"),
    (2, "HR", "Chicago"),
    (3, "Finance", "San Francisco"),
    (4, "Marketing", "Boston")  # No employees yet
])

# 4. Insert sample Employee data
cursor.executemany("INSERT INTO Employee (ID, NAME, SALARY, DEPT_ID) VALUES (?, ?, ?, ?)", [
    (1, "Alice", 120000, 1),
    (2, "Bob", 110000, 1),
    (3, "Charlie", 90000, 2),
    (4, "Diana", 95000, 3),
    (5, "Ethan", 97000, 3)
])

conn.commit()

# 5. Query: Department name & number of employees (include unstaffed)
query = """
SELECT 
    d.NAME AS department_name,
    COUNT(e.ID) AS num_employees
FROM Department d
LEFT JOIN Employee e
    ON d.DEPT_ID = e.DEPT_ID
GROUP BY d.NAME
ORDER BY num_employees DESC, department_name ASC;
"""

cursor.execute(query)
results = cursor.fetchall()

# 6. Display results
for row in results:
    print(row)


('Engineering', 2)
('Finance', 2)
('HR', 1)
('Marketing', 0)


## Summary

In this lab, we tested our knowledge of SQL queries against some real-world interview questions!